# QC Report 

Inputs 
- Manifest 
    - SampleID, SampleType, Batch, Column, Row
- Coverage 
    - SampleID, OutputPostprocessing 

In [ ]:
# TODO: Write out notes into the jupyter notebook
# TODO: Make an interactive script that people can run through 
# TODO: Extract out the functions as a package 
# TODO: Make a script someone could run and just look at a pdf 
# TODO: Turn this into an R shiny app 
# TODO: Make sure that it works with multiple plates etc and plots format correctly
# TODO: Add option for well instead of row, column
# TODO: manifest could be tsv, csv...

## Setup

In [ ]:
# Load necessary libraries
library(ggplot2)
library(dplyr)
library(tidyr)
library(ggbeeswarm)
library(stringr)

## Load Data  

In [ ]:
# User input 
# Input path to results directory from Mad4hatter pipeline
results_dir <- "data/example_mad4hatter_pipeline_output_larger"
# Input path to manifest 
manifest_file <- "data/manifest_large_multibatch.csv" 

In [ ]:
# Input a threshold for the reads needed to classify a locus as having 'good' coverage
read_threshold <- 100
# Input a minimum Reads per ASV used as a filter for the positive control plot
x <- 1 
# Minimum Allele Frequency per ASV used as a filter for positive control plot 
y <- 0.2

In [ ]:
sample_coverage_file <- file.path(results_dir,"sample_coverage.txt")
amplicon_coverage_file <- file.path(results_dir,"amplicon_coverage.txt")
allele_data_path <- file.path(results_dir, "allele_data.txt")

In [ ]:
manifest <- read.csv(manifest_file)
sample_coverage <- read.csv(sample_coverage_file, sep="\t")
allele_data <- read.csv(allele_data_path, sep='\t')

In [ ]:
# Load coverage per amplicon and add on amplicon length
amplicon_coverage = read.delim(amplicon_coverage_file)

Files 
Inputs 
- manifest 
    - SampleID, Batch, Row, Column, Parasitemia 
- sample_coverage 
    - SampleID, Stage, Reads 
- amplicon_coverage 
    - SampleID, Locus, Reads, OutputDada2, OutputPostprocessing
- allele_data
    - SampleID, Locus, ASV, Reads, PseudoCIGAR
    - Used for positive control section

Created tables 
- sample_coverage_with_manifest 
    - sample coverage, reads made numeric and NA replaced with 0, made wide format and joined with manifest
    - used for primer dimer plot
- amplicon_coverage_with_manifest 
    - amplicon coverage merged with manifest
    - used for negative control section and to make table below
- summary_samples 
    - made from amplicon_coverage_with_manifest 
    - splits locus name and takes reaction (pool) from that (reaction1 : 1A+5, reaction2 : 2.)
    - groups based on SampleID, batch, sampletype and reaction
    - For each it calculates the total reads output from the pipeline for that reaction, the the number with>100 reads
    - It calcs the prop of loci for each reaction by dividing by the number of loci in each, this is hardcoded
    - Used for balancing plot, but only uses SampleID,Batch,Reads,SampleType
    - Used for Amplicons with `good` read depth plot
    - Used for parasitemia plot
    - Used for plate maps

## Process input files

In [ ]:
merge_data <- function(manifest, sample_coverage) {
    merged_data <- sample_coverage %>%
        mutate(Reads = as.numeric(Reads)) %>% 
        replace_na(list(Reads = 0)) %>%
        pivot_wider(names_from = Stage, values_from = Reads) %>%
        left_join(manifest, by = "SampleID") 
  
    return(merged_data)
}

In [ ]:
# Reformat sample_coverage and add sample information from manifest 
sample_coverage_with_manifest <- merge_data(manifest, sample_coverage)

In [ ]:
# Add Columns to amplicons to specify experiment, SampleType
amplicon_coverage_with_manifest = amplicon_coverage %>%
  left_join(manifest,by = c("SampleID"))

In [ ]:
# Add reaction to amplicon table 
# reaction here refers to 1 of the 2 mPCR reactions reaction1 : 1A+5, reaction2 : 2.
# Note: in future versions of the pipeline this may have to come from the panel information, not the locus name 
amplicon_coverage_with_manifest <- amplicon_coverage_with_manifest %>%
  mutate(reaction = str_extract(Locus, "\\d(?=[A-Z]*$)"))

In [ ]:
# Create sample summary stats 
summary_samples <- amplicon_coverage_with_manifest %>%
  # Count unique loci per reaction
  group_by(reaction) %>%
  mutate(nreactionloci = n_distinct(Locus)) %>%
  ungroup() %>%
  # Calculate number of loci with reads over threshold per reaction
  group_by(SampleID, Batch, SampleType, reaction, nreactionloci) %>%
  summarize(
    reads_per_reaction = sum(OutputPostprocessing),
    n_good_loci = sum(OutputPostprocessing > read_threshold),
    .groups = "drop"
  ) %>%
    group_by(SampleID, Batch, SampleType) %>%
      mutate(
        reads_per_sample = sum(reads_per_reaction),
        # n100 = sum(n100.reaction),
        # n10 = sum(n10.reaction)
      ) %>%
  ungroup() %>%
  mutate(prop_good_loci = n_good_loci / nreactionloci) %>%
  left_join(manifest, by = join_by(SampleID, Batch, SampleType))

## Plotting Setup

In [ ]:
sample_colours <- c(
  "negative" = "red3",
  "positive" = "blue3",
  "sample" = "darkgrey",
  "NA" = "white"
)

## Primer Dimer Plot 

In [ ]:
# Function to generate the plot
generate_dimer_plot <- function(sample_coverage_with_manifest, sample_colours) {
    dimer_plot <- ggplot(data = sample_coverage_with_manifest) +
      # Plot points with different colors for each sample type
      geom_point(aes(x = Input + 0.9, 
                     y = (1 - OutputPostprocessing / Input) * 100, 
                     color = SampleType),
                 stroke = 1) +
    # Log scale for the x-axis
    scale_x_log10() +
    # Faceting by Batch column
    facet_wrap(~Batch) +
    # Adding labels and title
    ylab("% Dimers") +
    xlab("Input Reads") +
    ggtitle("Dimer Content")+
    # Color scale for SampleType
    scale_color_manual(values = sample_colours) 
    print(dimer_plot)
}

In [ ]:
generate_dimer_plot(sample_coverage_with_manifest, sample_colours)

## Balancing Across Batches 

* Only include if there are multiple experiments
* using postprocessing reads
* experiment means plate/batch (Group of samples processed at same time by same person -> pooled)

In [ ]:
generate_balancing_plot <- function(summary_samples, sample_colours) {
    balancing_plot <- ggplot() + ggbeeswarm::geom_quasirandom(
        data = summary_samples %>% 
        select(SampleID,Batch,reads_per_sample,SampleType) %>% 
        distinct(), 
        aes(x=Batch,y=reads_per_sample+0.9,color=SampleType))+
    scale_y_log10()+
    scale_color_manual(values = sample_colours) 
    ggtitle("Balancing Across Batches")
    print(balancing_plot)
}

In [ ]:
generate_balancing_plot(summary_samples, sample_colours)

## Amplicon Total reads v. number of targets that successfully amplified (e.g. >100 reads)
TODO: over 100 reads should be changed to a threshold that someone can set/ change
TODO: the reaction/pool needs to be fixed here. Could be pulled from panel information
* split by batch and pool
* colour controls
* Remove species loci 
* 2 thresholds set for reprep and repool - should be able to change and add these to the plot  
* output table with samples that failed for each category. Include total reads for rebalancing

List of samples to reprep/ repool 
* 50-75 repool 
* <50 reprep 
* Filters are done by each pool - table of which samples failed for which pools
* Include number of reads so you can rebalance what you put into next run

In [ ]:
# Function to generate the plot
generate_reads_by_amplification_success_plot <- function(summary_samples, threshold, sample_colours) {
    amp_plot <- ggplot(data = summary_samples) +
      # Plot points with different colors for each sample type
      geom_point(aes(x = reads_per_sample + 0.9, 
                     y = n_good_loci, 
                     color = SampleType),
                 stroke = 1) +
    # Log scale for the x-axis
    scale_x_log10() +
    # Faceting by Batch column
    facet_grid(cols = vars(Batch),rows = vars(reaction),
               scale = "free_y")+
    # Adding labels and title
    ylab(paste0("Amplicons with >",threshold," reads")) +
    xlab("Total Reads for Sample") +
    theme(axis.text.x = element_text(angle = 90, hjust = 1))+

    ggtitle("Amplicons with `good` read depth")+
    # Color scale for SampleType
    scale_color_manual(values = sample_colours) 
    print(amp_plot)
}

In [ ]:
generate_reads_by_amplification_success_plot(summary_samples, read_threshold, sample_colours)

## Parasitemia 
* Only include if parasitemia is in the manifest 
* parasitemia v. loci > 100 reads (should be interactive threshold)
* coloured by batch

In [ ]:
# Function to generate the plot
generate_parasitemia_by_amplification_success_plot <- function(summary_samples, threshold, sample_colours) {
    parasitemia_plot <- ggplot(data = summary_samples) +
      # Plot points with different colors for each sample type
      geom_point(aes(x = Parasitemia + 0.9, 
                     y = n_good_loci, 
                     color = SampleType),
                 stroke = 1) +
    # Log scale for the x-axis
    scale_x_log10() +
    # Faceting by Batch column
    facet_grid(cols = vars(Batch),rows = vars(reaction),
               scale = "free_y")+
    # Adding labels and title
    ylab(paste0("Amplicons with >",threshold," reads")) +
    xlab("Parasitemia (log10)") +
    theme(axis.text.x = element_text(angle = 90, hjust = 1))+

    ggtitle("Amplicons with `good` read depth")+
    # Color scale for SampleType
    scale_color_manual(values = sample_colours) 
    print(parasitemia_plot)
}

In [ ]:
generate_parasitemia_by_amplification_success_plot(summary_samples, read_threshold, sample_colours)

## Maps 
### Plate layouts by batch - well or row and column 

In [ ]:
create_plate_template <- function(batches, nrows=8, ncols=12) {
    quadrants <-
      expand.grid(
        Batch = batches,
        y = 1:nrows,
        x = 1:ncols
      ) %>%
      mutate(
        ymin = y - 0.45,
        ymax = y + 0.45,
        xmin = x - 0.45,
        xmax = x + 0.45
      ) %>%
      mutate(Row = rev(letters[1:nrows])[y]) %>%
      left_join(
        summary_samples %>%  select(Batch, Column, Row, SampleType) %>% distinct(),
        by = c("Batch", "x" = "Column", "Row" = "Row")
      )
    return(quadrants)
}

In [ ]:
quadrants <- create_plate_template(unique(summary_samples$Batch))

In [ ]:
plot_plate_layout <- function(summary_samples, sample_colours) {
    plate_layout <- ggplot(summary_samples) +
      # Tiles representing sample types, fill based on SampleType
      geom_tile(aes(x = Column, y = Row, fill = SampleType), color = "white") +
      # Facet by Batch
      facet_wrap(vars(Batch), scales = "free", ncol = 1) +
      # Quadrants as rectangles with borders
      geom_rect(
        data = quadrants,
        aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax),
        color = "black", fill = NA, linewidth = 1
      ) +
      # Customizing the x-axis
      scale_x_continuous(breaks = 1:12, labels = as.character(1:12)) +
    
      # Custom fill colors for each SampleType
      scale_fill_manual(
        values = sample_colours
      ) +
      # Theming
      theme_minimal() +
      theme(
        axis.text = element_text(size = 12),  
        strip.text = element_text(size = 14, face = "bold"), 
        panel.grid = element_blank(),  
        plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
      ) +
      # Title for the plot
    ggtitle("Sample Types and Quadrants Distribution by Batch")
    print(plate_layout)
}

In [ ]:
plot_plate_layout(summary_samples, sample_colours)

### Total number of reads per reaction per plate - log scale 

In [ ]:
plot_plate_with_feature <- function(summary_samples, sample_colours, fill_param, scale_midpoint) {
    plate_heatmap <- ggplot(summary_samples) +
      geom_tile(aes(
        x = Column,
        y = Row,
        fill = !!rlang::parse_expr(fill_param),
        color = SampleType  # Add color aesthetic for SampleType
      ), linewidth = 1.2) +  # Adjust size for outline thickness
    
      scale_fill_gradient2(
        low = "black",
        mid = "darkorange4",
        high = "darkorange",
        midpoint = scale_midpoint,
        name = "log10(reads)"
      ) +
      scale_color_manual(
          values = sample_colours
        ) + 
      facet_grid(rows = vars(reaction), cols = vars(Batch)) +
      geom_rect(
        data = quadrants,
        aes(
          xmin = xmin,
          xmax = xmax,
          ymin = ymin,
          ymax = ymax,
        ),
        fill = NA,
        linewidth = 0.3,
        color = "black"
      ) +

        # Theming
          theme_minimal() +
          theme(
            axis.text = element_text(size = 12),  
            strip.text = element_text(size = 14, face = "bold"), 
            panel.grid = element_blank(),  
            plot.title = element_text(size = 16, face = "bold", hjust = 0.5),
          ) +
      theme(legend.position = "bottom", aspect.ratio = 0.66)
    print(plate_heatmap)
}

In [ ]:
plot_plate_with_feature(summary_samples, sample_colours, "log10(reads_per_reaction + 0.1)", 2)

### Number of amplicons with >100 reads (0-1)

In [ ]:
plot_plate_with_feature(summary_samples, sample_colours, "prop_good_loci", 0.5)

## Positive Controls 
* are pos controls polyclonal
* x=sample, y=nlocus colour by mono and poly
* bottom right slide 14
* allow filter to be added  

In [ ]:
# Add on within sample allele freqeucny 
allele_data <- allele_data %>%
  group_by(SampleID, Locus) %>%
  mutate(AlleleFreq = Reads / sum(Reads)) %>%
  ungroup()

In [ ]:
pos_control_allele_data <- allele_data %>%
  inner_join(manifest %>% filter(SampleType == "positive"), by = "SampleID")

In [ ]:
# Define allele_col
allele_col <- "PseudoCIGAR"  # This can be changed dynamically

if (allele_col=="PseudoCIGAR") {
  pos_control_data <- pos_control_allele_data %>%
      group_by(SampleID, Locus, PseudoCIGAR) %>%
      summarise(Reads = sum(Reads), AlleleFreq = sum(AlleleFreq), .groups = "drop")
} else {
  pos_control_data <- pos_control_allele_data
}

In [ ]:
# Define thresholds
# Identify ASVs meeting both thresholds
locus_summary <- pos_control_data %>%
  group_by(SampleID, Locus) %>%
  summarise(
    NumASVs_Meeting_Threshold = sum(Reads > x & AlleleFreq > y),
    .groups = "drop"
  ) %>%
  mutate(Meets_Multi_ASV_Criteria = NumASVs_Meeting_Threshold > 1)

# Count loci per sample that meet and don't meet the threshold
result <- locus_summary %>%
  group_by(SampleID) %>%
  summarise(
    loci_poly = sum(Meets_Multi_ASV_Criteria),  # Loci with >1 ASV meeting the threshold
    loci_mono = sum(!Meets_Multi_ASV_Criteria)  # Loci that don't
  ) %>%
  pivot_longer(cols = starts_with("Loci_"), names_to = "Category", values_to = "Count") %>%
  mutate(Category = recode(Category, 
                           "loci_poly" = "Polyclonal", 
                           "loci_mono" = "Monoclonal"))

# Create stacked bar plot
ggplot(result, aes(x = SampleID, y = Count, fill = Category)) +
  geom_bar(stat = "identity", position = "stack") +
  labs(title = "Clonality of Loci for Positive Controls", x = "Sample ID", y = "Number of Loci") +
  theme_minimal() +
  scale_fill_manual(values = c("Polyclonal" = "steelblue", "Monoclonal" = "gray70"))


## Negative Controls 
* reads per locus per negative control by batch
* Overall plot as scatter plot
    * reads v. n loci with that n reads
    * colour by pool
    * split into sep plots for batch
    * if any locus for sample has over x reads plot as reads per locus for sample by pool  

In [ ]:
# TODO: colour by pool 
# TODO: decide on label for x axis 
# TODO: Only create second plot for samples that had a locus with over x reads at a locus

In [ ]:
# retrieve controls from amplicons.full
amplicons_negative = amplicon_coverage_with_manifest %>% 
  filter(SampleType=="negative") %>% 
  mutate(negative = paste0(Batch," Well: ",toupper(Row),Column)) 

In [ ]:
ggplot(amplicons_negative) +
  geom_histogram(aes(x = OutputPostprocessing))+#, fill = Pool)) +
  facet_wrap(~ negative) +
  xlim(-ifelse(
    max(amplicons_negative$OutputPostprocessing) == 0,
    1e2,
    max(amplicons_negative$OutputPostprocessing)
  ) / 50, ifelse(
    max(amplicons_negative$OutputPostprocessing) == 0,
    1e2,
    max(amplicons_negative$OutputPostprocessing)
  )) +
  xlab("Reads") +
  ylab("Count")

In [ ]:
ggplot(amplicons_negative) +
  geom_point(aes(x = Locus, y = Reads)) + #, color = Pool)) +
  facet_wrap(~ negative) +
  xlab("Coordinates")